# YOLOv8 학습 노트북

이 노트북은 아래처럼 구성된 데이터셋으로 YOLOv8을 학습합니다.

data_root/
  bboxes.json
  classes.txt
  images/
  labels/

가정:
- labels/ 에 YOLO 포맷 라벨이 있습니다 (class x_center y_center width height, 정규화).
- labels/ 가 비어 있고 bboxes.json 이 COCO 형식이면 변환 셀로 라벨을 생성할 수 있습니다.

데이터 경로가 다르면 아래 DATA_ROOT 를 수정하세요.


In [ ]:
# ultralytics 설치가 필요하면 아래를 실행하세요.
# %pip install -U ultralytics

from pathlib import Path
import json
import random


In [ ]:
DATA_ROOT = Path("..").resolve()  # notebooks/ 기준 repo 루트
BBOXES_JSON = DATA_ROOT / "bboxes.json"
CLASSES_TXT = DATA_ROOT / "classes.txt"
IMAGES_DIR = DATA_ROOT / "images"
LABELS_DIR = DATA_ROOT / "labels"

print("DATA_ROOT:", DATA_ROOT)
print("images 존재 여부:", IMAGES_DIR.exists(), "labels 존재 여부:", LABELS_DIR.exists())


In [ ]:
if CLASSES_TXT.exists():
    class_names = [line.strip() for line in CLASSES_TXT.read_text().splitlines() if line.strip()]
else:
    class_names = []

print("클래스 목록:", class_names)


## 선택 사항: COCO bboxes.json -> YOLO 라벨 변환

labels/ 가 비어 있고 bboxes.json 이 COCO 형식일 때만 실행하세요.


In [ ]:
from collections import defaultdict

label_files = list(LABELS_DIR.rglob("*.txt")) if LABELS_DIR.exists() else []
if label_files:
    print(f"라벨 파일이 {len(label_files)}개 있어 변환을 건너뜁니다.")
else:
    if not BBOXES_JSON.exists():
        print("labels/ 가 비어 있고 bboxes.json 도 없습니다. 라벨을 제공하거나 이 셀을 수정하세요.")
    else:
        data = json.loads(BBOXES_JSON.read_text())
        is_coco = isinstance(data, dict) and all(k in data for k in ("images", "annotations", "categories"))
        if not is_coco:
            raise ValueError("bboxes.json 이 COCO 형식이 아닙니다. 스키마에 맞게 이 셀을 수정하세요.")

        coco_categories = {c["id"]: c["name"] for c in data["categories"]}
        if not class_names:
            class_names = [coco_categories[k] for k in sorted(coco_categories)]
            CLASSES_TXT.write_text("\n".join(class_names))
            print("COCO 카테고리로 classes.txt 를 생성했습니다.")

        name_to_idx = {name: i for i, name in enumerate(class_names)}
        missing = [n for n in coco_categories.values() if n not in name_to_idx]
        if missing:
            print("경고: classes.txt 에 없는 카테고리:", missing)

        image_info = {img["id"]: img for img in data["images"]}
        ann_by_image = defaultdict(list)
        for ann in data["annotations"]:
            if ann.get("iscrowd"):
                continue
            ann_by_image[ann["image_id"]].append(ann)

        LABELS_DIR.mkdir(parents=True, exist_ok=True)
        for img_id, info in image_info.items():
            file_name = info.get("file_name")
            if not isinstance(file_name, str):
                continue
            img_rel = Path(file_name)
            if img_rel.parts and img_rel.parts[0] == "images":
                img_rel = Path(*img_rel.parts[1:])

            width = info.get("width")
            height = info.get("height")
            if not width or not height:
                continue

            lines = []
            for ann in ann_by_image.get(img_id, []):
                bbox = ann.get("bbox")
                if not bbox or len(bbox) != 4:
                    continue
                x, y, w, h = bbox
                if w <= 0 or h <= 0:
                    continue

                x_c = (x + w / 2) / width
                y_c = (y + h / 2) / height
                w_n = w / width
                h_n = h / height

                cat_name = coco_categories.get(ann.get("category_id"))
                if cat_name not in name_to_idx:
                    continue
                cls_id = name_to_idx[cat_name]

                lines.append(f"{cls_id} {x_c:.6f} {y_c:.6f} {w_n:.6f} {h_n:.6f}")

            label_path = LABELS_DIR / img_rel.with_suffix(".txt")
            label_path.parent.mkdir(parents=True, exist_ok=True)
            label_path.write_text("\n".join(lines))

        print("labels/ 에 YOLO 라벨을 생성했습니다:", LABELS_DIR)


## 학습/검증 분할 및 data.yaml 생성


In [ ]:
IMAGE_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}
train_dir = IMAGES_DIR / "train"
val_dir = IMAGES_DIR / "val"
train_txt = DATA_ROOT / "train.txt"
val_txt = DATA_ROOT / "val.txt"

if train_dir.exists() and val_dir.exists():
    train_spec = train_dir.resolve().as_posix()
    val_spec = val_dir.resolve().as_posix()
elif train_txt.exists() and val_txt.exists():
    train_spec = train_txt.resolve().as_posix()
    val_spec = val_txt.resolve().as_posix()
else:
    image_paths = [p for p in IMAGES_DIR.rglob("*") if p.suffix.lower() in IMAGE_EXTS]
    image_paths = sorted(image_paths)
    if not image_paths:
        raise FileNotFoundError(f"이미지 파일을 찾지 못했습니다: {IMAGES_DIR}")

    random.seed(42)
    random.shuffle(image_paths)
    split = int(len(image_paths) * 0.8)
    train_paths = image_paths[:split]
    val_paths = image_paths[split:]

    train_txt.write_text("\n".join(p.resolve().as_posix() for p in train_paths))
    val_txt.write_text("\n".join(p.resolve().as_posix() for p in val_paths))

    train_spec = train_txt.resolve().as_posix()
    val_spec = val_txt.resolve().as_posix()

print("train 경로:", train_spec)
print("val 경로:", val_spec)


In [ ]:
DATA_YAML = DATA_ROOT / "data.yaml"
names_yaml = json.dumps(class_names)
data_yaml_text = "\n".join([
    f"path: {DATA_ROOT.as_posix()}",
    f"train: {train_spec}",
    f"val: {val_spec}",
    f"nc: {len(class_names)}",
    f"names: {names_yaml}",
])
DATA_YAML.write_text(data_yaml_text)
print(DATA_YAML.read_text())


## YOLOv8 학습


In [ ]:
from ultralytics import YOLO
import torch

# GPU/CUDA 사용 가능 여부 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

model = YOLO("yolov8n.pt")
results = model.train(
    data=str(DATA_YAML),
    epochs=100,
    imgsz=640,
    batch=16,
    device=device,  # GPU/CUDA 사용
    project=str(DATA_ROOT / "runs"),
    name="yolov8_train",
    amp=True,  # Automatic Mixed Precision (GPU 성능 향상)
)

## 검증 및 내보내기


In [ ]:
metrics = model.val(data=str(DATA_YAML), device=device)
metrics

# ONNX로 내보내기
model.export(format="onnx")

## 샘플 이미지 추론


In [ ]:
sample_images = [p for p in IMAGES_DIR.rglob("*") if p.suffix.lower() in IMAGE_EXTS]
if sample_images:
    pred = model.predict(source=str(sample_images[0]), conf=0.25, device=device, save=True)
    print("예측 결과 저장 위치:", pred[0].save_dir)